In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt 
from py3gpp import *
import csv

In [2]:
## IQ Input files

input_file = '/home/tiwat/workarea/IQ_constellation/iq_python/input/IQDataFile_SPEEDTEST_10MS.csv' # PSS NID2=1
# input_file = '/home/tiwat/workarea/IQ_constellation/iq_python/input/IQDataFile_SPEEDTEST.csv'
# input_file = '/home/chatchamon/workarea/IQ_constellation/iq_python/input/IQDataFile_SPOTIFY.csv'  # PSS NID2=1
# input_file = '/home/chatchamon/workarea/IQ_constellation/iq_python/input/IQDataFile_SPOTIFY2.csv'

In [3]:
## read csv input file
with open(input_file, 'r') as f:
	reader = csv.reader(f)

	for _ in range(19):			# skip header
		next(reader)
	
	iq_val = []
	for row in reader:
		i_val = float(row[0])
		q_val = float(row[1])
		iq_val.append(complex(i_val, q_val))		# I+jQ
	
	waveform = np.array(iq_val[:230401])			# 10 ms
	# waveform = np.array(iq_val)			

print("len(waveform)", len(waveform))

len(waveform) 230401


In [4]:
delta_f = 0				# frequency offset
mu = 1					# for SCS
f = 1					# signal modulation

In [38]:
# scs = 30        # not sure
# syncNfft = 256 # minimum FFT size to cover SS burst
# syncSR = syncNfft * scs * 1e3     # move down, after get Nfft
nrbSSB = 20

dec_factor = 8			# decimated factor, to downsample
waveform = scipy.signal.decimate(waveform, dec_factor, ftype='fir') 
waveform /= max(waveform.real.max(), waveform.imag.max()) # scale max amplitude to 1
fs = 23.04e6 // dec_factor		# sampling rate 23.04 MHz, to 2.88 MSPS

In [39]:
waveform =  waveform * np.exp(-1j*2*np.pi*delta_f/fs*(np.arange(len(waveform))))
np.random.seed(69) # to get reproducible noise
noise = ((np.random.rand(waveform.shape[0]) - 0.5) + 1j*(np.random.rand(waveform.shape[0]) - 0.5))*0.8
SNR = 10*np.log10((np.linalg.norm(waveform) / np.linalg.norm(noise)))
waveform += noise
print(f'SNR = {SNR} dB')
# print("waveform w/ noise:",waveform)
print("len(waveform):", len(waveform))

SNR = 3.8032259979415137 dB
len(waveform): 57


In [40]:
carrier = nrCarrierConfig(NSizeGrid = 20, SubcarrierSpacing = 15 * 2**mu)
info = nrOFDMInfo(carrier)
Nfft = info['Nfft']     # Nfft = 512 for scs 30 kHz

In [41]:
pssIndices = np.arange((119-63), (119+64))      # same with nrPSSIndices()
# refGrid = np.zeros((nrbSSB*12, 2))
# print(pssIndices.shape, refGrid.shape)
 
## Frequency offset and PSS search
searchBW = 30*3       # don't know why
fshifts = np.linspace(-searchBW, searchBW, num=7) * 1e3 /2  # Half subcarrier step
print("fshifts", fshifts)

fshifts [-45000. -30000. -15000.      0.  15000.  30000.  45000.]


In [42]:
scsSSB = 15 * 2**(mu)
nSlot = 0
rxSampleRate = fs

In [43]:
# peak_value = np.zeros((fshifts.size, 3))
# peak_index = np.zeros((fshifts.size, 3))
# t = (np.arange(len(waveform)) / fs).reshape(-1, 1)
t = (np.arange(len(waveform)) / fs).reshape(-1)

In [11]:
print(f"waveform.shape: {waveform.shape}, t.shape: {t.shape}")

waveform.shape: (28801,), t.shape: (28801, 1)


In [44]:
index = np.zeros(3, 'int')
value = np.zeros(3)
peak_fIdx = np.zeros([len(fshifts), 3, ])


fIdx = 0
# for fIdx in np.arange(len(fshifts)):
coarseFrequencyOffset = fshifts[fIdx]
rxWaveformFreqShift = waveform * np.exp(-1j*2*np.pi*coarseFrequencyOffset*t)


# fig, axs = plt.subplots(3, 1, facecolor='w', sharex=True, sharey=True)
NID2 = 1
# for NID2 in np.arange(3, dtype='int'):

refGrid = np.zeros((nrbSSB*12))
refGrid[pssIndices] = nrPSS(NID2)
nSlot = 0

[refWaveform, info] = nrOFDMModulate(carrier, refGrid, SampleRate = fs)
refWaveform = refWaveform[info['CyclicPrefixLengths'][0]:]; # remove CP

timingOffset = nrTimingEstimate(waveform = rxWaveformFreqShift, nrb = nrbSSB, scs = scsSSB, initialNSlot = nSlot, refGrid = refWaveform, SampleRate = rxSampleRate)
print("rxWaveformFreqShift.shape", rxWaveformFreqShift.shape, "NID2",NID2, f"timingOffset", timingOffset)

temp = scipy.signal.correlate(rxWaveformFreqShift[timingOffset:], refGrid,'valid')  # correlate over 25 ms
# print(f"temp.shape",temp.shape)
print("temp.shape", temp.shape)

index[NID2]  = np.argmax(np.abs(temp))
value[NID2] = np.abs(temp[index[NID2]])
t_corr = np.arange(temp.shape[0])/fs*1e3
axs[NID2].plot(t_corr, np.abs(temp))



TypeError: unsupported operand type(s) for //: 'numpy.complex128' and 'int'

In [24]:
fs

2880000.0

In [173]:
index = np.zeros(3, 'int')
value = np.zeros(3)
peak_fIdx = np.zeros([len(fshifts), 3, ])


# fIdx = 0
# for fIdx in np.arange(len(fshifts)):
    # coarseFrequencyOffset = fshifts[fIdx]
    # rxWaveformFreqCorrected = waveform * np.exp(-1j*2*np.pi*coarseFrequencyOffset*t)

    fig, axs = plt.subplots(3, 1, facecolor='w', sharex=True, sharey=True)
    for NID2 in np.arange(3, dtype='int'):
        refGrid = nrResourceGrid(carrier)
        refGrid = refGrid[:, 0]
        refGrid[pssIndices] = nrPSS(NID2)
        nSlot = 0

        # [refWaveform, info] = nrOFDMModulate(carrier, refGrid, SampleRate = fs)
        # refWaveform = refWaveform[info['CyclicPrefixLengths'][0]:]; # remove CP

        timingOffset = nrTimingEstimate(waveform = rxWaveformFreqShift, nrb = nrbSSB, scs = scsSSB, initialNSlot = nSlot, refGrid = refGrid, SampleRate = rxSampleRate)
        # print(f"timingOffset", timingOffset)

        temp = scipy.signal.correlate(rxWaveformFreqShift[timingOffset:int(25e-3 * fs)], refGrid,'valid')  # correlate over 25 ms
        # print(f"temp.shape",temp.shape)

        index[NID2]  = np.argmax(np.abs(temp))
        value[NID2] = np.abs(temp[index[NID2]])
        t_corr = np.arange(temp.shape[0])/fs*1e3
        axs[NID2].plot(t_corr, np.abs(temp))
    peak_value = np.max(np.abs(temp))
    peak_index = index[np.argmax(value)]
    select_NID2 = np.argmax(value)
    # print("value", value, "index", index)
    # print("peak_value", peak_value, "select_NID2", select_NID2, "peak_index", peak_index)
# peak_fIdx = [fIdx, select_NID2, peak_index, peak_value]



IndentationError: unexpected indent (1514073515.py, line 11)

In [47]:

for fIdx in np.arange(len(fshifts)):
    coarseFrequencyOffset = fshifts[fIdx]
    rxWaveformFreqCorrected = waveform * np.exp(-1j*2*np.pi*coarseFrequencyOffset*t)
    # rxWaveformDS = scipy.signal.resample(rxWaveformFreqCorrected, syncSR, rxSampleRate) # no downsample, syncSR = rxSampleRate = fs

    # fig, axs = plt.subplots(3, 1, facecolor='w', sharex=True, sharey=True)    # for plotting

    for NID2 in [0,1,2]:
        refGrid = nrResourceGrid(carrier)
        refGrid = refGrid[:, 0]
        refGrid[pssIndices] = nrPSS(NID2)
        nslot = 0

        timingOffset = nrTimingEstimate(waveform = rxWaveformFreqCorrected, nrb = nrbSSB, scs = scsSSB, initialNSlot = nSlot, refGrid = refGrid, SampleRate = rxSampleRate)

        # rxGrid = nrOFDMDemodulate(waveform = rxWaveformFreqCorrected[timingOffset:][:np.min((len(waveform), (2048*8)//dec_factor))], nrb = nrbSSB, scs = scsSSB, initialNSlot = nSlot, SampleRate=rxSampleRate, CyclicPrefixFraction=0.5)

        # # correlate rxGrid with refGrid, as py3gpp_nrTimingEstimate not directly give correlation
        # temp = scipy.signal.correlate(rxGrid[:int(25e-3 * fs)], refGrid, 'valid')  # correlate over 25 ms
        
        # peak_index[NID2] = np.argmax(np.abs(temp))
        # peak_value[NID2] = np.abs(temp[peak_index[NID2]])
        # t_corr = np.arange(temp.shape[0])/fs*1e3



ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 240 and the array at index 1 has size 72